In [2]:
!pip install langchain-google-genai langchain-core langgraph pydantic google-generativeai --quiet
!pip install -q -U google-genai python-dotenv google-generativeai

In [ ]:
# Setting up LLM
import os
from dotenv import load_dotenv
from google import genai

load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

def llm_call(prompt: str) -> str:
    response = client.models.generate_content(
        model="gemini-1.5-flash", 
        contents=prompt
    )
    return response.text

# if __name__ == "__main__":
#     print(llm_call("Hello, Gemini!"))

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [12]:
# Setting up Gmail
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# 1. SETUP GMAIL AUTHENTICATION
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None
    if os.path.exists('contents/token.json'):
        creds = Credentials.from_authorized_user_file('contents/token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('contents/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0, open_browser=False) # Modified to not open browser
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()

# Check if gmail is connected
profile = service.users().getProfile(userId='me').execute()
print("Email:", profile["emailAddress"])

Email: forsignins043@gmail.com


In [13]:
def fetch_recent_emails(service, max_results=5):
    results = service.users().messages().list(
        userId="me",
        maxResults=max_results
    ).execute()

    messages = results.get("messages", [])
    emails = []

    for msg in messages:
        message = service.users().messages().get(
            userId="me",
            id=msg["id"],
            format="full"
        ).execute()

        snippet = message.get("snippet", "")
        emails.append(snippet)

    return emails